In [2]:
# Pip install (recommended)

!pip install ultralytics

In [5]:
!nvidia-smi

Sat Apr 20 19:42:25 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.86                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   45C    P8              4W /   75W |    1441MiB /   4096MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [8]:
train_path_img = "./arduino_car_dataset/images/train/"
train_path_label = "./arduino_car_dataset/labels/train/"
val_path_img = "./arduino_car_dataset/images/val/"
val_path_label = "./arduino_car_dataset/labels/val/"
test_path = "./arduino_car_dataset/test"

In [9]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### that is the directory that contains the images and there labels in which train, val, and test will get their data from
train_test_split("yolov8/data/") ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 1337 images---


  0%|          | 0/1070 [00:00<?, ?it/s]

------ Training data created with 80% split 1070 images -------


  0%|          | 0/267 [00:00<?, ?it/s]

------ Testing data created with a total of 267 images ----------
------ TASK COMPLETED -------


In [10]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.47  Python-3.9.19 torch-2.2.2+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
Setup complete  (20 CPUs, 15.8 GB RAM, 260.4/637.2 GB disk)


In [11]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current Directory:", current_directory)


Current Directory: C:\Users\aliye


In [12]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [14]:
!yolo task=detect mode=train model=yolov8s.pt data=yolov8/dataset.yaml epochs=100 imgsz=640 batch=8 project=yolov8/training_results name=car

^C


In [4]:
!yolo mode=export model=yolov8/training_results/car7/weights/best.pt format=onnx

Ultralytics YOLOv8.1.37 ðŸš€ Python-3.9.19 torch-2.2.2 CPU (12th Gen Intel Core(TM) i7-12700H)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'yolov8\training_results\car7\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...





   ---------------------------------------- 0.0/14.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.4 MB 1.3 MB/s eta 0:00:12
   ---------------------------------------- 0.0/14.4 MB 1.3 MB/s eta 0:00:12
   ---------------------------------------- 0.1/14.4 MB 1.1 MB/s eta 0:00:14
   ---------------------------------------- 0.1/14.4 MB 901.1 kB/s eta 0:00:16
   ---------------------------------------- 0.1/14.4 MB 944.1 kB/s eta 0:00:16
    --------------------------------------- 0.3/14.4 MB 1.4 MB/s eta 0:00:10
   - ----------------------------------

In [1]:
!yolo task=detect mode=predict model=best.pt conf=0.7 source="C:\Users\aliye\OneDrive\Desktop\Car.mp4"

Ultralytics YOLOv8.1.37 ðŸš€ Python-3.9.19 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (frame 1/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 80.1ms
video 1/1 (frame 2/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 10.0ms
video 1/1 (frame 3/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 29.7ms
video 1/1 (frame 4/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 10.0ms
video 1/1 (frame 5/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 20.0ms
video 1/1 (frame 6/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 19.9ms
video 1/1 (frame 7/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 9.9ms
video 1/1 (frame 8/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 10.0ms
video 1/1 (frame 9/1091) C:\Users\aliye\OneDrive\Desktop\Car.mp4: 384x640 2 cars, 10.0ms
video 1